In [17]:
import numpy as np
import ainsworth
from tqdm import tqdm
import matplotlib.pyplot as plt
np.set_printoptions(linewidth=1000)

In [ ]:
x,y = 96,96
HH = np.load('HH.npy')[:x,:y]
HV = np.load('HV.npy')[:x,:y]
VH = np.load('VH.npy')[:x,:y]
VV = np.load('VV.npy')[:x,:y] 

In [10]:
window_size = 8
HH_cal = []
HV_cal = []
VH_cal = []
VV_cal = []

M_list = []

pbar = tqdm(total = int((HH.shape[0] * HH.shape[1])/ window_size), desc = 'Calculating Parameters', unit = ' windows')

for a,b,c,d in ainsworth.sliding_window(HH, HV, VH, VV, window_size):
    cov = ainsworth.get_covariance(a,b,c,d)
    M = ainsworth.ainswoth_cal(cov, max_iter = 20, tolerance = 1e-10)
    M_list.append(M)
    pbar.update(1)
pbar.close()

Calculating Parameters: 100%|██████████| 1152/1152 [00:01<00:00, 674.81 windows/s]


In [ ]:
count = 0
i = 0

for a,b,c,d in ainsworth.sliding_window(HH, HV, VH, VV, window_size = 1):
    observed = np.array([a,b,c,d]).reshape((4,1))
    true = ainsworth.calibrate(observed,M_list[i])
    HH_cal.append(true[0,0])
    HV_cal.append(true[1,0])
    VH_cal.append(true[2,0])
    VV_cal.append(true[3,0])
    count += 1
    if count % 8 == 0:
        i += 1
    if i >= (x * x / window_size): break

In [ ]:
HH_cal = np.array(HH_cal).reshape(HH.shape)
HV_cal = np.array(HV_cal).reshape(HH.shape)
VH_cal = np.array(VH_cal).reshape(HH.shape)
VV_cal = np.array(VV_cal).reshape(HH.shape)